In [ ]:
import music21  
import numpy as np
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import pickle
from collections import defaultdict
import pandas as pd

In [ ]:
def read_midi(file_path): 
    notes = [] 
    notes_to_parse = None 
    key = 0

    try:
        midi= music21.converter.parse(file_path)
        key = midi.analyze('key')
        s2 = music21.instrument.partitionByInstrument(midi)
        for part in s2:
            if 'Piano' in str(part): 
                notes_to_parse = s2.recurse()
                for element in notes_to_parse:
                #select elements of only piano
                    if isinstance(element, music21.note.Note): 
                        notes.append(str(element.pitch))
                    elif isinstance(element, music21.chord.Chord): 
                        notes.append('.'.join(str(n) for n in element.pitches))
        return np.array(notes)
    except: 
        return np.array([])


def unique(np_array, array_type, unique_set): 
    unique_set_reverse = {n:idx for (idx, n) in unique_set.items()}
    new_array =[] 
    if array_type == 'x':
        for group in np_array: 
            temp = [] 
            for n in group: 
                temp.append(unique_set_reverse[n])
            new_array.append(temp)
    else: 
        for n in np_array: 
            new_array.append(unique_set_reverse[n])
    new_array = np.array(new_array)
    return new_array

def remove_rare(note_array, note_count, min_count): 
    note_array = [i for i in note_array if note_count[i] >= min_count]
    return np.array(note_array)

def get_tts(base_dir, pick_name = None, sample_num= 1000, timestep = 50, min_count = 20): 
    song_files = [f'{base_dir}/{i}' for i in os.listdir(base_dir)][:sample_num]
    songs_notes = [read_midi(i) for i in tqdm(song_files, desc = 'Reading Midi Files')]
    songs_notes = [i for i in songs_notes if len(i) != 0]
    all_notes = [i for note_ in songs_notes for i in note_]
    note_counter = defaultdict(int)
    for i in all_notes: 
        note_counter[i]+= 1
    #remove rare words
    songs_notes = [remove_rare(i, note_counter, min_count  = min_count) for i in songs_notes]
    all_notes = [i for note_ in songs_notes for i in note_]
    note_counter = defaultdict(int)
    for i in all_notes: 
        note_counter[i]+= 1
    all_notes_dict = {idx: n for idx, n in enumerate(set(all_notes))}
    x = [] 
    y = [] 

    pbar = tqdm(songs_notes, desc = 'Creating Timeseries')
    for idx_song, song in enumerate(pbar):
        for idx in range(timestep,len(song)-(timestep), 1): 
            x.append(song[idx:idx+timestep])
            y.append(song[idx + (timestep)])

    x = np.array(x)
    y = np.array(y)  

    new_x = unique(x, unique_set = all_notes_dict, array_type = 'x')
    new_y = unique(y, unique_set = all_notes_dict, array_type ='y')
    
    
    x_train, x_test, y_train, y_test = train_test_split(new_x, new_y, train_size = .92, random_state = 10)
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1],1)
    x_test = x_test.reshape(x_test.shape[0], x_test.shape[1],1)
    
    if pick_name:
        pick_tup= (x_train, x_test, y_train, y_test, all_notes_dict)
        pickle.dump(pick_tup, open(f'../Pickles/{pick_name}', 'wb'), protocol = 4)
                    
    return x_train, x_test, y_train, y_test, all_notes_dict

x_train, x_test, y_train, y_test, all_notes_dict = get_tts('../Classical', sample_num = 2000, pick_name= 'Classical1_1.p', 
                                                           min_count = 5, timestep = 20)

In [ ]:
from keras.layers import LSTM, Dense, Conv1D, Dropout, GlobalMaxPool1D
from keras.models import Sequential, load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import pickle

In [ ]:
model_type = 'Classical1_1'
def get_pickles(pick_name): 
    x_train, x_test, y_train, y_test, notes_dict = pickle.load(open(f'../Pickles/{pick_name}', 'rb'))
    
    return x_train, x_test, y_train, y_test, notes_dict


x_train, x_test, y_train, y_test, notes_dict = get_pickles(f'{model_type}.p')
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

In [ ]:
def get_lstm(n): 
    model = Sequential()
    model.add(LSTM(256, return_sequences = True))
    model.add(LSTM(256))    
#     model.add(Dense(128, activation = 'relu'))
#     model.add(Dropout(.3))
    model.add(Dense(512, activation = 'relu'))
#     model.add(Dense(512, activation = 'relu'))

    model.add(Dropout(.3))
    model.add(Dense(n, activation = 'softmax')) 
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', verbose = 1, patience=10, min_delta = .00075)
model_checkpoint = ModelCheckpoint(f'ModelWeights/LSTM_{model_type}.h5', verbose = 1, save_best_only=True,
                                  monitor = 'val_loss')

batch = 32 
epochs = 1000 
callbacks = [early_stopping, model_checkpoint]

unique_n = len(notes_dict)
lstm_model = get_lstm(unique_n)

#if you want to train from before
# lstm_model = load_model(f'ModelWeights/LSTM_{model_type}.h5')

lstm_history = lstm_model.fit(x_train, y_train, batch_size = batch, epochs = epochs, validation_data = (x_test, y_test), 
                        callbacks = callbacks)

pickle.dump(lstm_history, open(f'ModelPerf/{model_type}.p', 'wb'))

In [ ]:
%%javascript
IPython.notebook.save_notebook()

In [ ]:

import os
os.system(r'%windir%\system32\rundll32.exe powrprof.dll,SetSuspendState Hibernate')

In [ ]:
assert False

In [ ]:
#WITHOUT INSTRUMENT2
def read_midi(file_path): 
    notes = [] 
    notes_to_parse = None 
    key = 0

    try:
        midi= music21.converter.parse(file_path)
        key = midi.analyze('key')
        s2 = music21.instrument.partitionByInstrument(midi)
        for part in s2:
            if 'Piano' in str(part): 
                notes_to_parse = s2.recurse()
                for element in notes_to_parse:
                #select elements of only piano
                    if isinstance(element, music21.note.Note): 
                        notes.append(str(element.pitch))
                    elif isinstance(element, music21.chord.Chord): 
                        notes.append('.'.join(str(n) for n in element.pitches))
        return np.array(notes)
    except: 
        raise
        return np.array([])


file = f'../midi_songs/FFIX_Piano.mid'

read_midi(file)